# Benchmarking pip vs uv

**Ziel:**
Vergleiche die Installationsgeschwindigkeit und Ressourcennutzung von `pip install` vs. `uv pip sync` in einem kleinen Beispielprojekt.

**Abhängigkeiten:** numpy, pandas, click, pytest

In [1]:
!pip install --user hyperfine

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/48.8 kB ? eta -:--:--
   ---------------------------------------- 48.8/48.8 kB ? eta 0:00:00
   ---------------------------------------- 0.0/376.6 kB ? eta -:--:--
   ------------------ -------------------- 174.1/376.6 kB 10.2 MB/s eta 0:00:01
   ---------------------------------------  368.6/376.6 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 376.6/376.6 kB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   --------------- ------------------------ 0.9/2.3 MB 18.7 MB/s eta 0:00:01
   --------------------------------- ------ 2.0/2.3 MB 25.3 MB/s eta 0:00:01
   ------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# Unix-Umgebungsvariablen setzen
%%bash
# Erstelle requirements.txt
cat > requirements.txt << 'EOF'
numpy
pandas
click
pytest
EOF

echo 'requirements.txt erstellt:'
cat requirements.txt

In [3]:
# Windows-Umgebungsvariablen setzen
# Zelle als “Code” (Python-Kernel)
requirements = ["numpy", "pandas", "click", "pytest"]

with open("requirements.txt", "w") as f:
    f.write("\n".join(requirements))

print("requirements.txt erstellt:")
print(open("requirements.txt").read())


requirements.txt erstellt:
numpy
pandas
click
pytest


In [ ]:
# Nur auf Unix
%%bash
# Benchmark: Kalte Installation
hyperfine \
  --warmup 2 \
  --prepare 'rm -rf venv-pip && python3 -m venv venv-pip' \
  'venv-pip/bin/pip install -r requirements.txt' \
  --prepare 'rm -rf venv-uv && python3 -m venv venv-uv && pip install uv' \
  'venv-uv/bin/uv pip sync requirements.txt' \
  --export-csv cold_installs.csv

In [ ]:
# Nur auf Unix
%%bash
# Benchmark: Warme Installation
hyperfine \
  --warmup 1 \
  --prepare 'rm -rf venv-pip && python3 -m venv venv-pip && venv-pip/bin/pip install -r requirements.txt' \
  'venv-pip/bin/pip install -r requirements.txt' \
  --prepare 'rm -rf venv-uv && python3 -m venv venv-uv && pip install uv && venv-uv/bin/uv pip sync requirements.txt' \
  'venv-uv/bin/uv pip sync requirements.txt' \
  --export-csv warm_installs.csv

In [ ]:
# Nur auf Unix
%%bash
# Vergleich der Speicherbelegung
du -sh venv-pip venv-uv > disk_usage.txt

echo 'Festplattennutzung:'
cat disk_usage.txt

In [ ]:
# Unix code 
import pandas as pd
import matplotlib.pyplot as plt

# CSVs einlesen
cold = pd.read_csv('cold_installs.csv')
warm = pd.read_csv('warm_installs.csv')

# Kalte Installation
plt.figure()
plt.bar(['pip','uv'], cold['mean'])
plt.ylabel('Zeit (s)')
plt.title('Kalte Installationszeit')
plt.show()

# Warme Installation
plt.figure()
plt.bar(['pip','uv'], warm['mean'])
plt.ylabel('Zeit (s)')
plt.title('Warme Installationszeit')
plt.show()

In [6]:
# Widnows Zelle ersetzen (Code, Python-Kernel)

import subprocess, shutil, time
from pathlib import Path

def time_cmd(cmd, shell=False):
    """Führt `cmd` aus und gibt die Dauer in Sekunden zurück."""
    start = time.perf_counter()
    subprocess.run(cmd, shell=shell, check=True)
    return time.perf_counter() - start

# 1) Kalte Installation mit pip
if Path("venv-pip").exists():
    shutil.rmtree("venv-pip")
time_cmd(["python", "-m", "venv", "venv-pip"])
t_pip_cold = time_cmd(["venv-pip\\Scripts\\pip", "install", "-r", "requirements.txt"])

# 2) Kalte Installation mit uv
if Path("venv-uv").exists():
    shutil.rmtree("venv-uv")
time_cmd(["python", "-m", "venv", "venv-uv"])
# uv muss erst installiert werden:
time_cmd(["venv-uv\\Scripts\\pip", "install", "uv"])
t_uv_cold = time_cmd(["venv-uv\\Scripts\\uv", "pip", "sync", "requirements.txt"])

# 3) Warme Installationen
t_pip_warm = time_cmd(["venv-pip\\Scripts\\pip", "install", "-r", "requirements.txt"])
t_uv_warm = time_cmd(["venv-uv\\Scripts\\uv", "pip", "sync", "requirements.txt"])

# Ergebnisse ausgeben
print(f"Kalte Installation (pip): {t_pip_cold:.2f} s")
print(f"Kalte Installation (uv) : {t_uv_cold:.2f} s")
print(f"Warme Installation (pip): {t_pip_warm:.2f} s")
print(f"Warme Installation (uv) : {t_uv_warm:.2f} s")


Kalte Installation (pip): 43.30 s
Kalte Installation (uv) : 8.89 s
Warme Installation (pip): 1.41 s
Warme Installation (uv) : 0.03 s


In [8]:
# Zelle als “Code” mit Bash-Magic
!pip install pandas pytz python-dateutil

   ---------------------------------------- 0.0/229.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/229.9 kB ? eta -:--:--
   ------ -------------------------------- 41.0/229.9 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------  225.3/229.9 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 229.9/229.9 kB 2.0 MB/s eta 0:00:00


Error processing line 1 of C:\Users\abb2k\.conda\envs\env_name\Lib\site-packages\distutils-precedence.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 186, in addpackage
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored


In [11]:
import sys
!{sys.executable} -m pip install pandas pytz python-dateutil


Error processing line 1 of c:\Users\abb2k\.conda\envs\env_name\Lib\site-packages\distutils-precedence.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 186, in addpackage
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored


In [13]:
import subprocess, time, pandas as pd


ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

In [12]:
import subprocess, time, pandas as pd

def time_cmd(cmd):
    start = time.perf_counter()
    subprocess.run(cmd, check=True)
    return time.perf_counter() - start

results = []
for _ in range(5):  # fünf Wiederholungen
    # pip kalt
    subprocess.run(["rm", "-rf", "venv-pip"], shell=True)
    time_cmd(["python3", "-m", "venv", "venv-pip"])
    t_pip = time_cmd(["venv-pip/bin/pip", "install", "-r", "requirements.txt"])
    # pip warm
    t_pip_warm = time_cmd(["venv-pip/bin/pip", "install", "-r", "requirements.txt"])
    # uv kalt
    subprocess.run(["rm", "-rf", "venv-uv"], shell=True)
    time_cmd(["python3", "-m", "venv", "venv-uv"])
    subprocess.run(["venv-uv/bin/pip", "install", "uv"], check=True)
    t_uv = time_cmd(["venv-uv/bin/uv", "pip", "sync", "requirements.txt"])
    # uv warm
    t_uv_warm = time_cmd(["venv-uv/bin/uv", "pip", "sync", "requirements.txt"])
    
    results.append({
        "tool":"pip","mode":"cold","time":t_pip
    })
    results.append({
        "tool":"pip","mode":"warm","time":t_pip_warm
    })
    results.append({
        "tool":"uv","mode":"cold","time":t_uv
    })
    results.append({
        "tool":"uv","mode":"warm","time":t_uv_warm
    })

df = pd.DataFrame(results)
print(df.groupby(["tool","mode"])["time"].agg(["mean","std"]))


ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

## Ergebnisse und Fazit

